In [1]:
import pandas as pd
import numpy as np
import re             #re正则表达式 

一、数据获取 

In [2]:
data=pd.read_excel('李子柒螺蛳粉评论.xlsx')
print(data.shape)
print(data.info())

(2000, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
UserNick        2000 non-null object
comment_time    2000 non-null object
content         2000 non-null object
auctionSku      2000 non-null object
dtypes: object(4)
memory usage: 62.6+ KB
None


In [3]:
print(data.head())

UserNick         comment_time  \
0    g***o  2020-03-31 23:09:18   
1    a***黛  2020-04-01 10:37:51   
2    茉***儿  2020-04-01 11:35:05   
3    我***宝  2020-03-31 13:48:30   
4    t***4  2020-03-31 16:42:47   

                                             content auctionSku  
0  之前去旅游吃过一次螺狮粉以后，就一直喜欢吃螺狮粉，子柒家的螺狮粉的包装好精致啊，感觉像个大礼...  口味:螺蛳粉3袋装  
1  包装品质：包装挺严实的，三大袋，还送了一个鼠标垫 口感味道：口感我个人感觉比较清淡，但是该有...  口味:螺蛳粉3袋装  
2  包装品质：包装特别好，特别是包装袋的图腾和色彩搭配，看上去就很有食欲 #柒家美拍达人#   ...  口味:螺蛳粉3袋装  
3  虽然等了一个多月才等到发货，不过发货之后两天就收到了，快递也是很给力啊。红色的李子柒的外包装...  口味:螺蛳粉3袋装  
4  包装品质：非常中国风，也是柒家的特色 口感味道：不是很辣，对于不吃辣的本广东人来说可以接受。...  口味:螺蛳粉3袋装  


二、数据清洗
1.数据去重
2.删除多余列
3.新增字段：comment_time字段提取日期和年月字段
4.新增字段：content中提取情感字段

In [4]:
from collections import Counter
print(Counter(data.auctionSku))  #说明口味只有螺蛳粉3袋装

print(data.comment_time.max(),data.comment_time.min())  #评论的最早时间和最晚时间

Counter({'口味:螺蛳粉3袋装': 2000})
2020-04-02 23:59:13 2019-12-02 10:57:20


In [5]:
#1.数据去重+2.删除多余列
data.drop_duplicates(inplace=True)
data.drop(['auctionSku'], axis=1, inplace=True)
print(data.shape)

(1980, 3)


In [6]:
#3.新增字段：comment_time字段提取日期和年月字段
data['comment_time']=pd.to_datetime(data['comment_time'])
data['comment_date']=data['comment_time'].dt.date
data['comment_year_month']=data['comment_date'].apply(lambda x: str(x.year)+'-'+str(x.month))
print(data.head())

UserNick        comment_time  \
0    g***o 2020-03-31 23:09:18   
1    a***黛 2020-04-01 10:37:51   
2    茉***儿 2020-04-01 11:35:05   
3    我***宝 2020-03-31 13:48:30   
4    t***4 2020-03-31 16:42:47   

                                             content comment_date  \
0  之前去旅游吃过一次螺狮粉以后，就一直喜欢吃螺狮粉，子柒家的螺狮粉的包装好精致啊，感觉像个大礼...   2020-03-31   
1  包装品质：包装挺严实的，三大袋，还送了一个鼠标垫 口感味道：口感我个人感觉比较清淡，但是该有...   2020-04-01   
2  包装品质：包装特别好，特别是包装袋的图腾和色彩搭配，看上去就很有食欲 #柒家美拍达人#   ...   2020-04-01   
3  虽然等了一个多月才等到发货，不过发货之后两天就收到了，快递也是很给力啊。红色的李子柒的外包装...   2020-03-31   
4  包装品质：非常中国风，也是柒家的特色 口感味道：不是很辣，对于不吃辣的本广东人来说可以接受。...   2020-03-31   

  comment_year_month  
0             2020-3  
1             2020-4  
2             2020-4  
3             2020-3  
4             2020-3  


In [7]:
#自定义函数，长串字符串是否包含某些关键词
def contain_or_not(regex,word):
    return re.search(regex, word) is not None

In [8]:
#4.新增字段：content中提取情感字段
def judge_comment(content):

    # 创建一个空数据框
    judges = pd.DataFrame(np.zeros(13 * len(content)).reshape(len(content),13),
                      columns = ['品牌意识','物流正面','物流负面','包装正面','包装负面','日期正面','日期负面',
                      '原料正面','原料负面','口感正面','口感负面','性价比正面','性价比负面'])

    for i in range(len(content)):
        word = content[i]

        #根据评论中是否提到品牌关键字，判断消费者是否有品牌意识
        if contain_or_not('子柒|子七|小柒|小七',word):
            judges.iloc[i]['品牌意识'] = 1

        #判断物流相关的情感
        if contain_or_not('物流|快递|配送|取货|送货|收货',word):
            if contain_or_not('慢|龟速|暴力|一个月',word):
                judges.iloc[i]['物流负面'] = 1
            elif contain_or_not('快|迅速|速度',word):
                judges.iloc[i]['物流正面'] = 1

        #判断包装相关的情感
        if contain_or_not('包装|盒子|袋子|外观',word):
            if contain_or_not('破|瘪|简陋|漏气',word):
                judges.iloc[i]['包装负面'] = 1
            elif contain_or_not('高端|大气|精致|完整|严实|紧',word):
                judges.iloc[i]['包装正面'] = 1
        
        #判断日期相关的情感
        if contain_or_not('日期|时间|保质期|过期',word):
            if contain_or_not('短|过期|快到了',word):
                judges.iloc[i]['日期负面'] = 1
            elif contain_or_not('久|长|近',word):
                judges.iloc[i]['日期正面'] = 1

        #判断原料相关的情感
        if contain_or_not('粉|汤|料|腐竹|花生|豆角|酸笋',word):
            if contain_or_not('少|不多|不够|不新鲜',word):
                judges.iloc[i]['原料负面'] = 1
            elif contain_or_not('劲道|脆|多|足|饱|新鲜',word):
                judges.iloc[i]['原料正面'] = 1

        #判断口感相关的情感
        if contain_or_not('难吃|不好吃',word):
                judges.iloc[i]['口感负面'] = 1
        elif contain_or_not('好吃|鲜|香|食欲',word):
                judges.iloc[i]['口感正面'] = 1

        #判断性价比相关的情感
        if contain_or_not('不值|贵|亏了|不划算|不便宜|性价比低|不推荐',word):
                judges.iloc[i]['性价比负面'] = 1
        elif contain_or_not('划算|便宜|赚了|囤货|值|物美价廉|实惠|性价比高|不贵|推荐',word):
                judges.iloc[i]['性价比正面'] = 1

    return judges

# 得到带有情感的新数据
judge = judge_comment(data.content)
new_data = pd.concat([data,judge],axis = 1)
print(new_data.head())

UserNick        comment_time  \
0    g***o 2020-03-31 23:09:18   
1    a***黛 2020-04-01 10:37:51   
2    茉***儿 2020-04-01 11:35:05   
3    我***宝 2020-03-31 13:48:30   
4    t***4 2020-03-31 16:42:47   

                                             content comment_date  \
0  之前去旅游吃过一次螺狮粉以后，就一直喜欢吃螺狮粉，子柒家的螺狮粉的包装好精致啊，感觉像个大礼...   2020-03-31   
1  包装品质：包装挺严实的，三大袋，还送了一个鼠标垫 口感味道：口感我个人感觉比较清淡，但是该有...   2020-04-01   
2  包装品质：包装特别好，特别是包装袋的图腾和色彩搭配，看上去就很有食欲 #柒家美拍达人#   ...   2020-04-01   
3  虽然等了一个多月才等到发货，不过发货之后两天就收到了，快递也是很给力啊。红色的李子柒的外包装...   2020-03-31   
4  包装品质：非常中国风，也是柒家的特色 口感味道：不是很辣，对于不吃辣的本广东人来说可以接受。...   2020-03-31   

  comment_year_month  品牌意识  物流正面  物流负面  包装正面  包装负面  日期正面  日期负面  原料正面  原料负面  \
0             2020-3   1.0   0.0   0.0   1.0   0.0   0.0   0.0   1.0   0.0   
1             2020-4   0.0   0.0   0.0   1.0   0.0   0.0   0.0   1.0   0.0   
2             2020-4   1.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   
3             2020-3   1.0   1.0   0.0   0.0   0.0   0.0   0.0   1.0  

三、数据可视化
1.content字段的词云分析
2.每日评论数量走势图
3.每月评论数量条形图
4.评论占比条形图

In [9]:
#词云分析

import jieba        
import jieba.analyse

contents=new_data['content'].str.cat(sep='。')

#增加词库
jieba.add_word('螺蛳粉')
jieba.add_word('李子柒')
jieba.add_word('子柒')
jieba.add_word('好欢螺')

#读取停用词
stop_words=[]
with open('chineseStopWords.txt','r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        stop_words.append(line.strip())
# 添加停用词
stop_words.extend(['一袋', '一包','hellip', 
                   '一点',  '哈哈哈', ])      

# 评论字段分词处理
word_num = jieba.analyse.extract_tags(contents,
                                      topK=100,
                                      withWeight=True,
                                      allowPOS=())

# 删除停用词
word_num_selected = []
for i in word_num:
    if i[0] not in stop_words:
        word_num_selected.append(i)

#分词数据框
key_words = pd.DataFrame(word_num_selected, columns=['words','num'])
key_words.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\pr\AppData\Local\Temp\jieba.cache
Loading model cost 1.128 seconds.
Prefix dict has been built successfully.


,words,num
0,味道,0.274209
1,螺蛳粉,0.238752
2,好吃,0.203947
3,包装,0.193856
4,口感,0.163156


In [10]:
# 词云图
#结论：从词云图上可看出，商品标题中最多的字眼是螺蛳粉/味道/好吃/包装/口感/新鲜度/鼠标垫/品质/腐竹/整体/评价/李子柒/好欢螺等等。说明消费者最关注的是螺蛳粉的味道、包装、口感，而且很多人都是很满意的。词云中我们除了看到李子柒品牌还能看到好欢螺品牌，这两个品牌都是大众比较认可的螺蛳粉牌子。鼠标垫之所以出现频率比较高，是因为李子柒旗舰店将它作为一个赠品赠送给购买者的。

from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType
from pyecharts import options as opts

#词云图
word1 = WordCloud(init_opts=opts.InitOpts(width='1350px', height='750px'))
word1.add("", [*zip(key_words.words, key_words.num)],
          word_size_range=[20, 200],
          shape=SymbolType.DIAMOND)
word1.set_global_opts(title_opts=opts.TitleOpts('评论词云图'),
                      toolbox_opts=opts.ToolboxOpts())

In [11]:
# 每日评论数量走势图
#从图中可以看出，3月下旬到4月初的评论数增长很快，原因可能是春节和疫情期间螺蛳粉的物流比较慢，消费者收到货普遍都要20天到一个月，所以评论也比较集中在3月下旬到4月初。

from pyecharts.charts import Line
from pyecharts import options as opts

#计算每日评论数
comment_num =new_data['comment_date'].value_counts().sort_index()

# 折线图
line1 = Line(init_opts=opts.InitOpts(width='1350px', height='750px'))
line1.add_xaxis(comment_num.index.tolist())
line1.add_yaxis('评论热度', comment_num.values.tolist(),
areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
label_opts=opts.LabelOpts(is_show=False))
line1.set_global_opts(title_opts=opts.TitleOpts(title='每日评论数量走势图'),
 toolbox_opts=opts.ToolboxOpts(),
visualmap_opts=opts.VisualMapOpts(max_=400))

In [12]:
# 每月评论数量条形图
#从图中可以看出，评论主要集中在3月和4月。而我们的源数据中4月份只有两天数据，两天有793条评论，说明评论的人越来越多了。

from pyecharts.charts import Bar
from pyecharts import options as opts

#计算每月评论数
comment_num =new_data['comment_year_month'].value_counts().sort_index()

# 条形图
bar1 = Bar(init_opts=opts.InitOpts(width='1350px', height='750px')) 
bar1.add_xaxis(comment_num.index.tolist())
bar1.add_yaxis('评论热度', comment_num.values.tolist()) 
bar1.set_global_opts(title_opts=opts.TitleOpts(title='每月评论数量条形图'),
                     visualmap_opts=opts.VisualMapOpts(max_=1200)) 

In [13]:
#评论占比分布图
#从条形图中我们看出，正面评论的占比相对较高，尤其是61%的评论都提到了口感正面。

# 计算占比
rank = new_data.iloc[:, 5:].sum().reset_index().sort_values(0, ascending=False) 
rank.columns = ['分类', '提及次数']
rank['占比'] = round(rank['提及次数'] / new_data.shape[0],2) 


# 条形图
bar2 = Bar(init_opts=opts.InitOpts(width='1350px', height='750px')) 
bar2.add_xaxis(rank['分类'].tolist())
bar2.add_yaxis('评论内容', rank['占比'].tolist()) 
bar2.set_global_opts(title_opts=opts.TitleOpts(title='评论内容占比'),
                     visualmap_opts=opts.VisualMapOpts(max_=0.6)) 

In [14]:
# 生成tab
from pyecharts.charts import Tab

tab = Tab()
tab.add(word1, '词云图')
tab.add(line1, '折线图')
tab.add(bar1, '条形图1')
tab.add(bar2, '条形图2')
tab.render('李子柒螺蛳粉评论分析可视化.html')

'D:\\Code\\Python\\李子柒螺蛳粉评论分析可视化.html'